## Попытка исправить падение показателей PRE алгоритма начиная с 15 релиза

    v15
    365 - {'profit': 262.15, 'profit_p': 8.15, 'pot_p': 0, 'config': RNFT+ 3/pre7:0/3 x l2 x 1.2¤ , 'op': 1543}
    
    last_test_date='2024-12-31',
    test_days_num=30,  # 365,
    
задача #281

In [1]:
%reload_ext autoreload
%autoreload 2

In [4]:
from bot.test import TestAlgorithm
from app.cache import LocalCache
from bot.vis import TaskProgress
from app.config import RunConfig

from concurrent.futures import ThreadPoolExecutor, as_completed

TICKER = 'RNFT'

test_configs = [
    (RunConfig(
        ticker=TICKER,
        step_max_cnt=max_shares,
        step_base_cnt=base_shares,
        step_lots=2,

        majority_trade=True,
        pretest_period=pretest_period,
        pretest_type=pretest_type,

        threshold_buy_steps=threshold_b_n,
        threshold_sell_steps=threshold_s_n,
        stop_up_p=stop_up_p,
        stop_down_p=stop_down_p,
        # start_time='07:01',

        step_size=step_size,
        step_set_orders_cnt=step_cnt,
    ))
    for max_shares in [3]
    for base_shares in [0]
    for threshold_b_n in [0]
    for threshold_s_n in [0]
    for step_size in [1.2]
    for step_cnt in [3]
    for pretest_period in [7]
    for pretest_type in [RunConfig.PRETEST_PRE]
    for stop_up_p in [0]
    for stop_down_p in [0]
]


def run_test(config: RunConfig):
    test_alg = TestAlgorithm(do_printing=False, config=config)
    return test_alg.test(
        last_test_date='2024-12-31',
        test_days_num=365,  # 365,
        shares_count=0,
    )

results = []
LocalCache.clear()
progress = TaskProgress(len(test_configs))

with ThreadPoolExecutor(max_workers=1) as executor:
    future_to_params = {executor.submit(run_test, config): config for config in test_configs}
    
    for future in as_completed(future_to_params):
        params = future_to_params[future]
        res = future.result()
        if res:
            results.append(res)
        progress.update_progress()

# Вывод результатов или их дальнейшая обработка
sorted_results = sorted(results, key=lambda x: float(x['profit_p']), reverse=True)

print()
for item in sorted_results:
    print(item)

Запуск в 18:03
Закончено в 18:04, количество 1, длительность 0:00:27                                               

{'exp': 'RNFT pre ', 'profit': -29126, 'profit_p': -29.13, 'profit_p_avg': -0.08, 'config': RNFT+ 4/pre7:0/3 x l2 x 1.2¤ , 'last_conf': RNFT+ 4/pre7:0/3 x l73 x 1.2¤ , 'op': 3513}


## Для PRE алгоритма

##### v14
    30 - {'profit': -54.1, 'profit_p': -4.16, 'config': RNFT+ 3/pre4:0/3 x l2 x 1.2¤ , 'op': 82}
    250 - {'profit': 192.7, 'profit_p': 7.12, 'config': RNFT+ 3/pre7:0/3 x l2 x 1.2¤ , 'op': 1338}
    365 - {'profit': 102.25, 'profit_p': 6.19, 'config': RNFT+ 3/pre5:0/3 x l2 x 1.2¤ , 'op': 1932}

#141 старт тестирования с 0-й минуты, а не с 1-й
aca054b1 a.sapozhkov <a@sapozhkov.ru> on 31.05.2024 at 16:25
падение вот из-за этого коммита

а для этого уже есть исследование в #142
и там противоречивые результаты. задача всё еще открыта

для старта в 07:01 выдает {'profit': 118.15, 'profit_p': 7.15, 'config': RNFT+ 3/pre7:0/3 x l2 x 1.2¤ , 'op': 1928}

##### v15
    30 - {'profit': -41.6, 'profit_p': -3.2, 'pot_p': 0, 'config': RNFT+ 3/pre7:0/3 x l2 x 1.2¤ , 'op': 86}
    250 - {'profit': 146.2, 'profit_p': 5.41, 'pot_p': 0, 'config': RNFT+ 3/pre7:0/3 x l2 x 1.2¤ , 'op': 1330}
    265 - {'profit': 262.15, 'profit_p': 8.15, 'pot_p': 0, 'config': RNFT+ 3/pre7:0/3 x l2 x 1.2¤ , 'op': 1543}
    365 - {'profit': -36.0, 'profit_p': -2.18, 'pot_p': 0, 'config': RNFT+ 3/pre7:0/3 x l2 x 1.2¤ , 'op': 1920}
    
#159 кэш для исторических прогонов - убрал вариации для рабочих/нерабочих дней
d49bf7dd a.sapozhkov <a@sapozhkov.ru> on 11.06.2024
вот после этого комита идет изменение результата 

вот из-за этого стало иначе. если перекомментировать, то вернется обратно. кэш на работу не влияет, но отрабатывает сильно быстрее

        # #162 #159 тип выбора предыдущих дней - рабочие/все. может пригодиться для тестирования/экспериментов
        # days_list = TickerCache.get_days_list(last_test_date, test_days_num)
        days_list = TickerCache.get_days_list_working_only(last_test_date, test_days_num)
        
это, кстати, еще 1 плюс, чтобы попробовать отдельную работу выходных и будних дней в #271


##### v16 - v21
    30 - {'profit': -79.55, 'profit_p': -5.21, 'pot_p': 0, 'config': RNFT+ 3/pre7:0/3 x l2 x 1.2¤ , 'op': 136}
    250 - {'profit': -36.0, 'profit_p': -2.25, 'pot_p': 0, 'config': RNFT+ 3/pre7:0/3 x l2 x 1.2¤ , 'op': 1920}
    365 - {'profit': 38.2, 'profit_p': 2.06, 'pot_p': 0, 'config': RNFT+ 3/pre7:0/3 x l2 x 1.2¤ , 'op': 2753}

##### v22 - v23
    -//-
    365 - {'profit': 38.2, 'profit_p': 2.06, 'profit_p_avg': 0.01, 'config': RNFT+ 3/pre7:0/3 x l2 x 1.2¤ , 
        'last_conf': RNFT+ 3/pre7:0/3 x l2 x 1.2¤ , 'op': 2753}

количество стало 3009
тут реально был поправлен баг - счетчик учитывает все заявки определенных типов, а в имитации был не тот ответ
исправлено в e3a1f059 в файле client_test_env.py

##### v24 - v26
    -//- подросло количество операций
    365 - {'profit': 38.2, 'profit_p': 2.06, 'profit_p_avg': 0.01, 'config': RNFT+ 3/pre7:0/3 x l2 x 1.2¤ , 
        'last_conf': RNFT+ 3/pre7:0/3 x l2 x 1.2¤ , 'op': 3009}

bd6b0f70 #252 переделал систему расчета прибыли (баг с ростом базового числа и оборота при этом)
тут реально баг был исправлен

##### v27
    30 - {'exp': 'RNFT pre ', 'profit': -7174, 'profit_p': -7.17, 'profit_p_avg': -0.24, 
        'config': RNFT+ 3/pre7:0/3 x l2 x 1.2¤ , 'last_conf': RNFT+ 3/pre7:0/3 x l2 x 1.2¤ , 'op': 156}
    250 - {'exp': 'RNFT pre ', 'profit': -3006, 'profit_p': -3.01, 'profit_p_avg': -0.01, 
        'config': RNFT+ 3/pre7:0/3 x l2 x 1.2¤ , 'last_conf': RNFT+ 3/pre7:0/3 x l2 x 1.2¤ , 'op': 2094}
    365 - {'exp': 'RNFT pre ', 'profit': 1044, 'profit_p': 1.04, 'profit_p_avg': 0.0, 
        'config': RNFT+ 3/pre7:0/3 x l2 x 1.2¤ , 'last_conf': RNFT+ 3/pre7:0/3 x l157 x 1.2¤ , 'op': 3009}

это первая - #148 расширение временного диапазона торгов до полного дня
3fb - 0:00:25 - {'exp': 'RNFT pre ', 'profit': -28885, 'profit_p': -28.89, 'profit_p_avg': -0.08, 'config': RNFT+ 3/pre7:0/3 x l2 x 1.2¤ , 'last_conf': RNFT+ 3/pre7:0/3 x l98 x 1.2¤ , 'op': 2786}

вот тут произошел провал, когда поменялось время работы

если в конфиге вот это вернуть, то становится 1% как было

    start_time='07:00',  # 10:00
    end_time='15:29',  # 18:29

подумать, может быть стоит оставить потом вернуть стандартный диапазон для PRE алгоритмов

хотя, конечно, 1% годовых, это далеко не то, ради чего стоит рисковать   

##### v28 (на кэше свечей из 29 версии)
    30 - {'exp': 'RNFT pre ', 'profit': -10418, 'profit_p': -10.42, 'profit_p_avg': -0.35, 
        'config': RNFT+ 4/pre7:0/3 x l2 x 1.2¤ , 'last_conf': RNFT+ 4/pre7:0/3 x l92 x 1.2¤ , 'op': 192}
    250 - {'exp': 'RNFT pre ', 'profit': -14831, 'profit_p': -14.83, 'profit_p_avg': -0.06, 
        'config': RNFT+ 4/pre7:0/3 x l2 x 1.2¤ , 'last_conf': RNFT+ 4/pre7:0/3 x l88 x 1.2¤ , 'op': 2463}
    365 - {'exp': 'RNFT pre ', 'profit': -29126, 'profit_p': -29.13, 'profit_p_avg': -0.08, 
        'config': RNFT+ 4/pre7:0/3 x l2 x 1.2¤ , 'last_conf': RNFT+ 4/pre7:0/3 x l73 x 1.2¤ , 'op': 3513}

##### v29 WIP

-//-
